# 📚 4.5相爱篇-模型的加载与保存

🔲能今天做好的事就不要等到明天。以梦为马，学习趁年华。

训练完网络后需要对训练结果进行保存，即网络参数的持久化。当需要使用网络进行推理预测时，加载保存的数据即可。

## 一、本节目标
        本节主要讲述tensorflow2中模型的类型、保存模型的方法、加载模型的方法

## 二、 模型格式
tensorflow2中模型有ckpt、h5、pb三种格式。  
（1）ckpt格式  
ckpt格式是对模型进行分开保存的，主要是3种文件：checkpoint、data、index，各文件的描述如下。  

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598641959821361615986418552.png"/>

（2）h5格式   
h5文件将模型的参数以及网络结构保存为一个整体文件。  
（3）pb格式  
pb格式服务器部署模型，谷歌推荐的保存模型的方式是保存模型为 PB 文件，它具有语言独立性，可独立运行，封闭的序列化格式，任何语言都可以解析它，它允许其他语言和深度学习框架读取、继续训练和迁移 TensorFlow 的模型。  

## 三、 模型保存

### 2.1 API整体介绍
     针对三种格式的模型，tensorflow2三种保存模型的方式，如下表所示。

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598656456577111615986563485.png"/>

### 2.2 API整体介绍

In [2]:
#导入库
import numpy as np
import tensorflow as tf
import os

In [3]:
#制作训练数据
x_train = np.random.random((1000, 32))
y_train = np.random.randint(10, size=(1000,))
#制作验证数据
x_val = np.random.random((200, 32))
y_val = np.random.randint(10, size=(200,))
#制作测试数据
x_test = np.random.random((200, 32))
y_test = np.random.randint(10, size=(200,))
#创建网络
inputs = tf.keras.Input(shape=(32,), name='digits')
x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = tf.keras.layers.Dense(10, name='predictions')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

#构建优化器及损失函数
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['sparse_categorical_accuracy']
model.compile(optimizer,loss ,metrics)

#训练网络
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))
model.summary()

Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 4s 4ms/sample - loss: 2.3147 - sparse_categorical_accuracy: 0.1020 - val_loss: 2.3235 - val_sparse_categorical_accuracy: 0.0900
Epoch 2/5
1000/1000 [==============================] - 0s 85us/sample - loss: 2.2932 - sparse_categorical_accuracy: 0.1240 - val_loss: 2.3205 - val_sparse_categorical_accuracy: 0.1100
Epoch 3/5
1000/1000 [==============================] - 0s 87us/sample - loss: 2.2844 - sparse_categorical_accuracy: 0.1410 - val_loss: 2.3198 - val_sparse_categorical_accuracy: 0.1150
Epoch 4/5
1000/1000 [==============================] - 0s 80us/sample - loss: 2.2745 - sparse_categorical_accuracy: 0.1460 - val_loss: 2.3134 - val_sparse_categorical_accuracy: 0.0950
Epoch 5/5
1000/1000 [==============================] - 0s 89us/sample - loss: 2.2663 - sparse_categorical_accuracy: 0.1670 - val_loss: 2.3230 - val_sparse_categorical_accuracy: 0.1050
Model: "model"
___________________

### 2.3 保存ckpt模型

方法1

In [4]:
os.mkdir("./models") 
checkpoint_dir="./models"
checkpoint_prefix=os.path.join(checkpoint_dir,"ckpt")
checkpoint=tf.train.Checkpoint(optimizer=optimizer)
checkpoint.save(file_prefix=checkpoint_prefix)

'./models\\ckpt-1'

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598771322774891615987712181.png"/>

方法二

In [5]:
model.save_weights('./model1/save_model.ckpt')

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598775004278801615987748994.png"/>

### 2.4 保存h5模型

In [6]:
model.save("tf_model.h5")
model.save_weights("tf_model_weights.h5")

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598779606227981615987795024.png"/>

### 2.5 保存pb模型

In [7]:
model_path="tf_model.pb"
tf.keras.models.save_model(model,model_path,overwrite=True,include_optimizer=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tf_model.pb\assets


<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598784515146571615987844103.png"/>

## 四、 模型加载
模型加载是模型用于实际的关键步骤，tensorflow2 提供了两种方式加载模型：只加载模型参数、同时加载模型参数和网络结构。其中只载入模型参数需要重新建立与模型完全一致的网络结构，同时加载模型参数和网络结构则不需要新建网络结构，使用较灵活。

### 4.1 总体

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161598738165534781615987380575.png"/>

### 4.2ckpt模型加载
加载checkpoint模型权重。

In [9]:
model.load_weights('./model1/save_model.ckpt')
print(type(model.predict(x_test)))

<class 'numpy.ndarray'>


### 4.3 h5模型加载
（1）同时加载网络结构和参数

In [10]:
model_load=tf.keras.models.load_model("tf_model.h5")
print(type(model_load.predict(x_test)))

<class 'numpy.ndarray'>


（2）只加载参数

In [11]:
model.load_weights("tf_model_weights.h5")
print(type(model.predict(x_test)))

<class 'numpy.ndarray'>
